In [1]:
import netaddr, time, sys, json, zipfile, csv
import pandas as pd
from zipfile import ZipFile
from io import TextIOWrapper

In [2]:
int(netaddr.IPAddress("34.67.75.25"))

574835481

In [3]:
df = pd.read_csv("ip2location.csv")

,low,high,code,region
0,0,16777215,-,-


In [4]:
def test(ip_address):
    result = []
    for ip in ip_address:
        t0 = time.time()
        result_dict = {}
        low = df[df["low"] <= int(netaddr.IPAddress(ip))].reset_index()
        high = low[df["high"] >= int(netaddr.IPAddress(ip))].reset_index().drop(columns=["level_0", "index"])
        ms = time.time() - t0
        result_dict["ip"] = ip
        result_dict["int_ip"] = int(netaddr.IPAddress(ip))
        result_dict["region"] = high["region"][0]
        result_dict["ms"] = ms
        result.append(result_dict)
    file = open("ips.txt", "w")
    string = str(result).replace("\'", "\"")
    file.write(string)
    file.close()
    return string
    
    

In [5]:
test(["1.1.1.2","9.9.9.9", "1.1.1.1", "1.1.1.2"])

/home/seanwells/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/home/seanwells/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/home/seanwells/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/home/seanwells/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


'[{"ip": "1.1.1.2", "int_ip": 16843010, "region": "United States of America", "ms": 0.006490468978881836}, {"ip": "9.9.9.9", "int_ip": 151587081, "region": "United States of America", "ms": 0.0046155452728271484}, {"ip": "1.1.1.1", "int_ip": 16843009, "region": "United States of America", "ms": 0.004470348358154297}, {"ip": "1.1.1.2", "int_ip": 16843010, "region": "United States of America", "ms": 0.0040705204010009766}]'

In [6]:
df = pd.read_csv("ip2location.csv")
ip = "34.67.75.25"

In [7]:
len(df)

174972

In [8]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]
def ip_test(ips, index=0, result=[]):
    while len(ips) > 0:
        ip = ips.pop(0)
        dic_t = {}
        t0 = time.time()
        for low in df["low"][index:]:
            if low <= int(netaddr.IPAddress(ip)):
                index += 1
        try:
            assert (df["high"][index-1] >= int(netaddr.IPAddress(ip)) and df["low"][index-1] <= int(netaddr.IPAddress(ip))), "int_ip not in range"
        except:
            ip_test([ip],0,result)
            continue
        dic_t["ip"] = ip 
        dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
        dic_t["region"] = df["region"][index-1] 
        dic_t["ms"] = (time.time() - t0) * 1000 
        result.append(dic_t)
        ip_test(ips, index-1, result)
    return result

In [9]:
ip_test(ips)

[{'ip': '1.1.1.1',
  'int_ip': 16843009,
  'region': 'United States of America',
  'ms': 801.7711639404297},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 930.0129413604736},
 {'ip': '9.9.9.9',
  'int_ip': 151587081,
  'region': 'United States of America',
  'ms': 808.7117671966553},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 801.4926910400391}]

In [10]:
ip_test(ips)

[{'ip': '1.1.1.1',
  'int_ip': 16843009,
  'region': 'United States of America',
  'ms': 801.7711639404297},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 930.0129413604736},
 {'ip': '9.9.9.9',
  'int_ip': 151587081,
  'region': 'United States of America',
  'ms': 808.7117671966553},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 801.4926910400391}]

In [11]:
def zip_csv_iter(name):
    with ZipFile(name) as zf:
        with zf.open(name.replace(".zip", ".csv")) as f:
            reader = csv.reader(TextIOWrapper(f))
            for row in reader:
                yield row

In [12]:
zip1 = "large.zip"
zip2 = "small.zip"

def sorting(instance):
    first_digits = int(instance[0].split(".")[0])
    second_digits = int(instance[0].split(".")[1])
    third_digits = int(instance[0].split(".")[2])
    return (first_digits, second_digits, third_digits)

def sampling(zip1, zip2, stride=30000):    
    reader = zip_csv_iter(zip1)
    header = next(reader) # the list of all column names
    header.append("region")
    ip_idx = header.index("ip")
    samples = []
    while True:
        row = next(reader)
        ip_str = ""
        for character in row[ip_idx]:
            if character.isalpha():
                ip_str += "0"
            else:
                ip_str += character 
        row.append(ip_test([ip_str])[-1]["region"])
        samples.append(row)
        for i in range(int(stride)-1):
            try:
                next(reader)
            except StopIteration:
                samples.sort(key=sorting)
                print(samples)
                with ZipFile(zip2, "w") as zf:
                    with zf.open(zip2.replace(".zip", ".csv"), "w") as raw:
                        with TextIOWrapper(raw) as f:
                            writer = csv.writer(f, lineterminator='\n')
                            writer.writerow(header) # write the row of column names to zip2
                            for row in samples:
                                writer.writerow(row) # write a row to zip2
                return

In [13]:
sampling("large.zip", "small_samp.zip", 30000)

[['52.45.218.ihf', '2017-01-01', '10:23:25', '0.0', '827156.0', '0001341522-06-000030', '.txt', '200.0', '5974.0', '0.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America'], ['54.212.94.jcd', '2017-01-01', '03:31:36', '0.0', '1461219.0', '0000000000-13-001261', '-index.htm', '301.0', '243.0', '1.0', '0.0', '1.0', '10.0', '0.0', '', 'United States of America'], ['104.197.32.ihd', '2017-01-01', '00:00:00', '0.0', '1111711.0', '0001193125-12-324016', '-index.htm', '200.0', '7627.0', '1.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America'], ['107.23.181.jje', '2017-01-01', '07:01:00', '0.0', '1330399.0', '0001299933-16-003371', 'edgar/data/1330399/-', '200.0', '7065.0', '1.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America'], ['107.178.195.bbb', '2017-01-01', '19:57:16', '0.0', '1584509.0', '0001584509-16-000514', 'armk-20160930_pre.xml', '200.0', '68682.0', '0.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America'], ['157.55.39.eja', '2017-01-01',

In [14]:
reader = zip_csv_iter("small_samp.zip")
header = next(reader)
for row in reader:
    print(row)

['52.45.218.ihf', '2017-01-01', '10:23:25', '0.0', '827156.0', '0001341522-06-000030', '.txt', '200.0', '5974.0', '0.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['54.212.94.jcd', '2017-01-01', '03:31:36', '0.0', '1461219.0', '0000000000-13-001261', '-index.htm', '301.0', '243.0', '1.0', '0.0', '1.0', '10.0', '0.0', '', 'United States of America']
['104.197.32.ihd', '2017-01-01', '00:00:00', '0.0', '1111711.0', '0001193125-12-324016', '-index.htm', '200.0', '7627.0', '1.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['107.23.181.jje', '2017-01-01', '07:01:00', '0.0', '1330399.0', '0001299933-16-003371', 'edgar/data/1330399/-', '200.0', '7065.0', '1.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['107.178.195.bbb', '2017-01-01', '19:57:16', '0.0', '1584509.0', '0001584509-16-000514', 'armk-20160930_pre.xml', '200.0', '68682.0', '0.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['157.55.39.eja', '2017-01-01', '15:0

In [69]:
reader = zip_csv_iter("small_samp.zip")
header = next(reader)
ip_idx = header.index("ip")
reg_idx = header.index("region")

for row in reader:
    print(row)

['52.45.218.ihf', '2017-01-01', '10:23:25', '0.0', '827156.0', '0001341522-06-000030', '.txt', '200.0', '5974.0', '0.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['54.212.94.jcd', '2017-01-01', '03:31:36', '0.0', '1461219.0', '0000000000-13-001261', '-index.htm', '301.0', '243.0', '1.0', '0.0', '1.0', '10.0', '0.0', '', 'United States of America']
['104.197.32.ihd', '2017-01-01', '00:00:00', '0.0', '1111711.0', '0001193125-12-324016', '-index.htm', '200.0', '7627.0', '1.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['107.23.181.jje', '2017-01-01', '07:01:00', '0.0', '1330399.0', '0001299933-16-003371', 'edgar/data/1330399/-', '200.0', '7065.0', '1.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['107.178.195.bbb', '2017-01-01', '19:57:16', '0.0', '1584509.0', '0001584509-16-000514', 'armk-20160930_pre.xml', '200.0', '68682.0', '0.0', '0.0', '0.0', '10.0', '0.0', '', 'United States of America']
['157.55.39.eja', '2017-01-01', '15:0

In [16]:
(9, 3) > (9, 2)

True

In [17]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]
def ip_test(ips, index=0, result=[]):
    while len(ips) > 0:
        ip = ips.pop(0)
        dic_t = {}
        t0 = time.time()
        try:
            assert (df["high"][index] >= int(netaddr.IPAddress(ip)) and df["low"][index] <= int(netaddr.IPAddress(ip))), "int_ip not in range"
            dic_t["ip"] = ip 
            dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
            dic_t["region"] = df["region"][index] 
            dic_t["ms"] = (time.time() - t0) * 1000 
            result.append(dic_t)
            ip_test(ips, index, result)
        except:
            for low in df["low"][index:]:
                if low <= int(netaddr.IPAddress(ip)):
                    index += 1
            try:
                assert (df["high"][index-1] >= int(netaddr.IPAddress(ip)) and df["low"][index-1] <= int(netaddr.IPAddress(ip))), "int_ip not in range"
            except:
                ip_test([ip],0,result)
                continue
            dic_t["ip"] = ip 
            dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
            dic_t["region"] = df["region"][index-1] 
            dic_t["ms"] = (time.time() - t0) * 1000 
            result.append(dic_t)
            ip_test(ips, index-1, result)
    return result

In [18]:
ip_test(ips)

[{'ip': '1.1.1.1',
  'int_ip': 16843009,
  'region': 'United States of America',
  'ms': 941.9224262237549},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 0.033855438232421875},
 {'ip': '9.9.9.9',
  'int_ip': 151587081,
  'region': 'United States of America',
  'ms': 829.3788433074951},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 801.3594150543213}]

In [19]:
#old ip_test
def ip_test(ips, index=0, result=[]):
    while len(ips) > 0:
        ip = ips.pop(0)
        dic_t = {}
        t0 = time.time()
        for low in df["low"][index:]:
            if low <= int(netaddr.IPAddress(ip)):
                index += 1
        try:
            assert (df["high"][index-1] >= int(netaddr.IPAddress(ip)) and df["low"][index-1] <= int(netaddr.IPAddress(ip))), "int_ip not in range"
        except:
            ip_test([ip],0,result)
            break
        dic_t["ip"] = ip 
        dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
        dic_t["region"] = df["region"][index-1] 
        dic_t["ms"] = (time.time() - t0) / 1000
        result.append(dic_t)
        ip_test(ips, index-1, result)
    return result

In [20]:
def ip_test(ips, index=0, result=list()):
    for ip in ips:
        dic_t = {}
        t0 = time.time()
        if (df["high"][index] >= int(netaddr.IPAddress(ip))) and (df["low"][index] <= int(netaddr.IPAddress(ip))):
            dic_t["ip"] = ip 
            dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
            dic_t["region"] = df["region"][index] 
            dic_t["ms"] = (time.time() - t0) * 1000
            result.append(dic_t)
        else:
            print("2.0")
            try:    
                for low in df["low"]:
                    if (low <= int(netaddr.IPAddress(ip))) and (df["high"].at[index] >= int(netaddr.IPAddress(ip))):
                        print("2")
                        dic_t["ip"] = ip 
                        dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
                        dic_t["region"] = df["region"][index] 
                        dic_t["ms"] = (time.time() - t0) * 1000
                        result.append(dic_t) 
                    else: 
                        index += 1
            except KeyError:
                ip_test([ip], 0, result)
    return result

In [21]:
ip_test(["1.1.1.1", "1.1.1.2", "9.9.9.9", "1.1.1.2"])

2.0
2
2.0
2
2
2
2
2
2
2
2
2
2
2


KeyError: 349932

In [ ]:
len(df)

In [ ]:
df["low"][174972]

In [25]:
def ip_test(ips, index=0, result=list()):
    dic_t = {}
    t0 = time.time()
    for ip in ips:
        while index <= 174972:
            try:
                assert (df["high"][index] >= int(netaddr.IPAddress(ip)) and low <= int(netaddr.IPAddress(ip))), "int_ip not in range"
                dic_t["ip"] = ip 
                dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
                dic_t["region"] = df["region"][index] 
                dic_t["ms"] = (time.time() - t0) * 1000
                result.append(dic_t)
                index += 1
            except:
                index += 1
    return result

In [26]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]
ip_test(ips, 0, [])

[]

In [32]:
df[2:]

,low,high,code,region
2,16777472,16778239,CN,China
3,16778240,16779263,AU,Australia
4,16779264,16781311,CN,China
5,16781312,16785407,JP,Japan
6,16785408,16793599,CN,China
...,...,...,...,...
174967,3758094336,3758095359,HK,Hong Kong
174968,3758095360,3758095871,CN,China
174969,3758095872,3758096127,SG,Singapore
174970,3758096128,3758096383,AU,Australia


In [54]:
def ip_test(ips, index=0, result=[]):
    while len(ips) > 0:
        ip = ips.pop(0)
        index %= len(df)
        info = {}
        t0 = time.time()
        print("BEFORE: ", ip, index)
        for low in df["low"][index:]:
            try:
                assert (low <= int(netaddr.IPAddress(ip))) and (df["high"].at[index] >= int(netaddr.IPAddress(ip)))
                info["ip"] = ip
                info["int_ip"] = int(netaddr.IPAddress(ip)) 
                info["region"] = df["region"][i] 
                info["ms"] = (time.time() - t0) * 1000
                result.append(info)
            except:
                index += 1
    return result

In [55]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]
ip_test(ips, 0, [])

BEFORE:  1.1.1.1 0
BEFORE:  1.1.1.2 0
BEFORE:  9.9.9.9 0
BEFORE:  1.1.1.2 0


[]

In [49]:
9 % 10

9

In [52]:
test = 9
test %= 2

In [53]:
test

1

In [56]:
def ip_test(ips, index=0, result=[]):
    while len(ips) > 0:
        ip = ips.pop(0)
        dic_t = {}
        t0 = time.time()
        try:
            assert (df["high"][index] >= int(netaddr.IPAddress(ip)) and df["low"][index] <= int(netaddr.IPAddress(ip))), "int_ip not in range"
            dic_t["ip"] = ip 
            dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
            dic_t["region"] = df["region"][index] 
            dic_t["ms"] = (time.time() - t0) * 1000 
            result.append(dic_t)
            ip_test(ips, index, result)
        except:
            for low in df["low"][index:]:
                if low <= int(netaddr.IPAddress(ip)):
                    index += 1
            try:
                assert (df["high"][index-1] >= int(netaddr.IPAddress(ip)) and df["low"][index-1] <= int(netaddr.IPAddress(ip))), "int_ip not in range"
            except:
                ip_test([ip],0,result)
                continue
            dic_t["ip"] = ip 
            dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
            dic_t["region"] = df["region"][index-1] 
            dic_t["ms"] = (time.time() - t0) * 1000 
            result.append(dic_t)
            ip_test(ips, index-1, result)
    return result

In [57]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]
ip_test(ips, 0, [])

[{'ip': '1.1.1.1',
  'int_ip': 16843009,
  'region': 'United States of America',
  'ms': 438.8093948364258},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 0.030040740966796875},
 {'ip': '9.9.9.9',
  'int_ip': 151587081,
  'region': 'United States of America',
  'ms': 426.0525703430176},
 {'ip': '1.1.1.2',
  'int_ip': 16843010,
  'region': 'United States of America',
  'ms': 426.76830291748047}]

In [66]:
def ip_test(ips, index=0, result=[]):
    while len(ips) > 0:
        ip = ips.pop(0)
        dic_t = {}
        index %= len(df)
        t0 = time.time()
        for low in df["low"][index:]:
            index += 1
            try:
                assert (df["high"][index-1] >= int(netaddr.IPAddress(ip)) and df["low"][index-1] <= int(netaddr.IPAddress(ip))), "int_ip not in range"
                dic_t["ip"] = ip 
                dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
                dic_t["region"] = df["region"][index-1] 
                dic_t["ms"] = (time.time() - t0) * 1000
                result.append(dic_t)
            except:
                index += 1
        dic_t["ip"] = ip 
        dic_t["int_ip"] = int(netaddr.IPAddress(ip)) 
        dic_t["region"] = df["region"][index-1] 
        dic_t["ms"] = (time.time() - t0) * 1000
        result.append(dic_t)
        ip_test(ips, index-1, result)
    return result

In [67]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]
ip_test(ips, 0, [])

KeyError: 349942